In [1]:
import os
import json
import tensorflow as tf
from numpy import array
import random
random.seed( 3 )

from six.moves.urllib.request import urlretrieve

SOURCE_URL = 'http://pwz.mit.edu/data/darkest/'
WORK_DIRECTORY = "/img-data/algo"

def maybe_download(filename):
    """A helper to download the data files if not present."""
    if not os.path.exists(WORK_DIRECTORY):
        os.mkdir(WORK_DIRECTORY)
    filepath = os.path.join(WORK_DIRECTORY, filename)
    if not os.path.exists(filepath):
        filepath, _ = urlretrieve(SOURCE_URL + filename, filepath)
        statinfo = os.stat(filepath)
        print('Succesfully downloaded', filename, statinfo.st_size, 'bytes.')
    else:
        print('Already downloaded', filename)
    return filepath  

In [47]:
interial_data = maybe_download('0-interial.txt')

print interial_data

with open(interial_data, 'r') as f0:
    print f0
    print f0.readline()
    print f0.readline()
    print f0.readline()
    print f0.readline()
    print f0.readline()

    int_d = json.load(f0)


('Succesfully downloaded', '0-interial.txt', 1095398, 'bytes.')
/img-data/dark/0-interial.txt
<open file '/img-data/dark/0-interial.txt', mode 'r' at 0x7fec0cfd58a0>
[[[1600.0, 1609.4345744325888, 1638.3026044712456, 1688.3814127936432, 1762.9388867540088, 1867.2890237742929, 2009.8152702556624, 2203.8704341837347, 2471.4765165886479, 2851.0992122866114, 3415.8223058683775, 4322.7102553933055, 5000.0000000000009, 5000.0000000000009, 5000.0, 5000.0, 4999.9999999999991, 5000.0, 5000.0000000000009, 4999.9999999999991, 5000.0, 5000.0, 5000.0, 5000.0, 4999.9999999999991, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0], [1600.0, 1609.434574432591, 1638.3026044712526, 1688.3814127936578, 1762.9388867540008, 1867.2890237743513, 2009.815270255514, 2203.8704341837183, 2471.4765165885196, 2851.0992122867347, 3415.8223058683152, 4322.7102553928944, 4999.9999999999991, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0000000000009,

ValueError: No JSON object could be decoded

In [4]:

interial_data = maybe_download('0-interial.txt')
exterial_data = maybe_download('1-exterial.txt')
outmost_data = maybe_download('2-outmost.txt')
test_data = maybe_download('test.txt')

with open(interial_data, 'r') as f0,open(exterial_data, 'r') as f1,open(outmost_data, 'r') as f2,open(test_data, 'r') as test:
    int_d = json.load(f0)
    ext_d = json.load(f1)
    out_d = json.load(f2)
    test_d = json.load(test)


    train_data = []
    train_label = []
    test_data = []

    for img in int_d:
        temp = []
        for row in img:
            for i in row:
                temp.append(i)
        train_data.append(temp)
        train_label.append([1.0,0,0])
    for img in ext_d:
        temp = []
        for row in img:
            for i in row:
                temp.append(i)
        train_data.append(temp)
        train_label.append([0,1.0,0])
    for img in out_d:
        temp = []
        for row in img:
            for i in row:
                temp.append(i)
        train_data.append(temp)
        train_label.append([0,0,1.0])
        
    for img in test_d:
        temp = []
        for row in img:
            for i in row:
                temp.append(i)
        test_data.append(temp)


    print len(train_data),len(train_label),len(test_data)

    x = tf.placeholder(tf.float32,[None,1800])
    W = tf.Variable(tf.zeros([1800, 3])) # now have 3 labels, each pixel to each label have a weight
    b = tf.Variable(tf.zeros([3])) #each label have one bias
    y = tf.nn.softmax(tf.matmul(x, W) + b)
    y_ = tf.placeholder(tf.float32, [None, 3]) # the placeholder for the right labels

    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(cross_entropy)

    init = tf.initialize_all_variables()
    sess = tf.InteractiveSession()

    sess.run(init)                


    def normalize(lst):
        return [i/5000 for i in lst]


    #print normalize(train_data[0])
    train_data = [normalize(i) for i in train_data]
    test_data = [normalize(i) for i in test_data]
    
    print array(train_data)
    print array(test_data)

    
    def subset(lst,index,num):
        if index+num<len(lst):
            return lst[index:index+num]
        else:
            lst0 = lst[index:]
            lst1 = lst[:(num-(len(lst)-index))]
            lst0.extend(lst1)
            return lst0
        
    
    
    setsize = 20
    setAll = 120
    
    print len(subset(train_data,110,setsize))
    
    for i in range(50000):
        myindex =  int(random.random()*setAll)
        if i%100 == 0:
            print i
        batch_xs, batch_ys = array(subset(train_data,myindex,setsize)), array(subset(train_label,myindex,setsize))
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
    #print array(train_data[0]),array(test_data[0])
    print sess.run(W)
    print sess.run(b)
    #correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    sys_prediction = y
    print sess.run(sys_prediction, feed_dict={x: array(test_data)})
    
    #print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))




            

('Already downloaded', '0-interial.txt')
('Already downloaded', '1-exterial.txt')
('Already downloaded', '2-outmost.txt')
('Already downloaded', 'test.txt')
120 120 40
[[ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]
 [ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]
 [ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]
 ..., 
 [ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]
 [ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]
 [ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]]
[[ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]
 [ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]
 [ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]
 ..., 
 [ 0.32        0.32188691  0.32766052 ...,  1.          1.          1.        ]
 [ 0.32        0.

In [5]:
def byteify(input):
    if isinstance(input, dict):
        return {byteify(key): byteify(value)
                for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [byteify(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

import numpy as np

In [6]:
print W.eval(sess)

out_0 = "["
out_1 = "["
out_2 = "["

for (x,y), value in np.ndenumerate(W.eval(sess)):
    
    if y == 0:
        out_0 += str(value)+","
        print x
    if y == 1:
        out_1 += str(value)+","
    if y == 2:
        out_2 += str(value)+","

out_0 += "]"
out_1 += "]"
out_2 += "]"


with open("weight_0.txt", "w") as text_file:
    text_file.write(out_0)
with open("weight_1.txt", "w") as text_file:
    text_file.write(out_1)
with open("weight_2.txt", "w") as text_file:
    text_file.write(out_2)


[[ 0.00418379 -0.00123156 -0.00295222]
 [ 0.00420842 -0.00123883 -0.0029696 ]
 [ 0.00428393 -0.00126105 -0.00302288]
 ..., 
 [ 0.01307417 -0.00384862 -0.00922571]
 [ 0.01307417 -0.00384862 -0.00922571]
 [ 0.01307417 -0.00384862 -0.00922571]]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
